<a href="https://colab.research.google.com/github/humaira1996/Colab/blob/main/FINAL_Copy_of_AVGword2vecwithCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
%matplotlib inline
import nltk
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import pandas as pd
import sqlite3
import re
import matplotlib.pyplot as plt
#import nbconvert
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/My Drive/SM_FRIEND_NEWSDATA

/gdrive/My Drive/SM_FRIEND_NEWSDATA


In [ ]:
data = pd.read_csv('Data_samitdata.csv').apply(lambda x: x.astype(str).str.lower())

In [ ]:
data.drop_duplicates(keep=False, inplace=True)

In [ ]:
stopwords = pd.read_csv('Stopwords.csv')

def to_encode(bangla_word):
    return bangla_word.encode('utf-8')


def to_decode(bangla_word):
    return bangla_word.decode('utf-8')

set_stop = set(stopwords['words'].map(to_encode))

In [ ]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()

texts = data['text'].values

In [ ]:
def cleanhtml(sentence):
    cleanr = re.compile('/.*?')
    cleantext = re.sub(cleanr,' ',sentence)
    return cleantext
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    cleaned = re.sub(r'[A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z]', r'', cleaned1)
    cleaned1 = re.sub(r'[1|2|3|4|5|6|7|8|9|0]', r'', cleaned)
    return cleaned1

In [ ]:
def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    if(to_encode(cleaned_word) not in set_stop):
                        s = stemmer.stem_word(cleaned_word)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

In [ ]:
X = pre_process(data['text'].values)
#y = data['label'].values

In [ ]:
Y = data['label'].values

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, shuffle=Flase)# this is for time series split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33,random_state=42) # this is random splitting
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33,random_state=42) # this is random splitting

In [ ]:
label=np.unique(y_train)

label

array(['eco', 'ent', 'int', 'sprt', 'state'], dtype=object)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
#vectorizer.fit(X_train) # fit has to happen only on train data

W2VEC MODEL

In [ ]:
i=0
list_of_sentance_train=[]
for sentance in X_train:
    list_of_sentance_train.append(sentance.split())

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
# this line of code trains your w2v model on the give list of sentances
w2v_model=Word2Vec(list_of_sentance_train,min_count=5,size=3000, workers=10)

In [ ]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  104608
sample words  ['স্বাধীনতা', 'দিবস', 'চেয়ারপারসন', 'খালেদা', 'জিয়াসহ', 'কারারুদ্ধ', 'নেতা', 'কর্মী', 'মুক্ত', 'আন্দোলন', 'শপথ', 'বিএনপিআজ', 'বেলা', 'সাড়', 'বিএনপির', 'প্রতিষ্ঠাতা', 'জিয়াউর', 'রহমান', 'সমাধি', 'শ্রদ্ধা', 'নিবেদন', 'শেষ', 'মহাসচিব', 'মির্জা', 'ফখরুল', 'ইসলাম', 'আলমগীর', 'কথা', 'জানানমির্জা', 'আজক', 'দিন', 'কারাগার', 'অবিলম্ব', 'গ্রহণ', 'মহান', 'উপলক্ষ', 'সকাল', 'রাজধানীর', 'চন্দ্রিমা', 'উদ্যান', 'জড়ো', 'বিপুলসংখ্যক', 'একপর্যা', 'জ্যেষ্ঠ', 'উপস্থিত', 'সবাই', 'ফুল', 'দুর্ভাগ্য', 'উদ্\u200cযাপন', 'কারাবরণ']


In [ ]:
from tqdm import tqdm
import numpy as np

CONVERT TRAINDATA TEXT

In [ ]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_train): # for each review/sentence
    sent_vec = np.zeros(3000) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors_train = np.array(sent_vectors)
#print(np.asarray(sent_vectors).shape)
#print(sent_vectors[0])

#print(np.asarray(X).shape)

100%|██████████| 167759/167759 [1:44:38<00:00, 26.72it/s]


CONVERTING CV DATA TEXT

In [ ]:
i=0
list_of_sentance_cv=[]
for sentance in X_cv:
    list_of_sentance_cv.append(sentance.split())

In [ ]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors_cv = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_cv): # for each review/sentence
    sent_vec = np.zeros(3000) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_cv.append(sent_vec)
sent_vectors_cv = np.array(sent_vectors_cv)
#print(np.asarray(sent_vectors_cv).shape)
#print(sent_vectors_cv[0])

100%|██████████| 82628/82628 [44:45<00:00, 30.77it/s]


CONVERTING TEST DATA TEXT

In [ ]:
i=0
list_of_sentance_test=[]
for sentance in X_test:
    list_of_sentance_test.append(sentance.split())

In [ ]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors_test = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_test): # for each review/sentence
    sent_vec = np.zeros(3000) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_test.append(sent_vec)
sent_vectors_test = np.array(sent_vectors_test)
#print(np.asarray(sent_vectors_test).shape)
#print(sent_vectors_test[0])

100%|██████████| 123325/123325 [1:04:52<00:00, 31.68it/s]


FIT WITH A MODEL 
# **GaussianNB classifier**

In [ ]:
from sklearn . naive_bayes import GaussianNB
from sklearn import metrics
from sklearn . metrics import accuracy_score
from sklearn . model_selection import cross_val_score

In [ ]:
Gmodel = GaussianNB() . fit ( sent_vectors , y_train )

 #neigh.fit(sent_vectors_train, y_train)

NameError: ignored

# *Cross validation for gaussian nb*

In [ ]:
#using k-fold

accuracy = cross_val_score(Gmodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10)
print(accuracy)

[0.84642382 0.85598451 0.84993344 0.85247489 0.8496914  0.84836016
 0.85550042 0.8513857  0.84979424 0.8540305 ]


In [ ]:
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

Accuracy of Model with Cross Validation is: 85.13579066288246


In [ ]:
#using cross_validate() function

from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
scores = cross_validate(Gmodel, sent_vectors_cv, y_cv, scoring=scoring)
sorted(scores.keys())

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

In [ ]:
scores['test_recall_macro']

array([0.88584986, 0.88404225, 0.88099244, 0.88975153, 0.88302936])

# *accuracy for test data*

In [ ]:
from sklearn . metrics import classification_report
y_test_pred  = Gmodel . predict ( sent_vectors_test )
y_cv_pred = Gmodel . predict(sent_vectors_cv)

In [ ]:
#print("Cross val score:",cross_val_score(Gmodel,df,data1.check,cv=5,scoring='accuracy'))
print ( 'Model accuracy %s' % accuracy_score ( y_cv , y_cv_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_cv , y_cv_pred , normalize = False ) )
print ( classification_report ( y_cv , y_cv_pred,target_names=label ) )

#X_labels . append ( "Gaussian" )
#y_labels . append ( accuracy_score ( y_test , y_cv_pred ) )

Model accuracy 0.8497966790918333
accuracy normalized 70217
              precision    recall  f1-score   support

         eco       0.45      0.90      0.60      4205
         ent       0.74      0.92      0.82      6778
         int       0.59      0.87      0.70      7052
        sprt       0.99      0.91      0.95     11227
       state       0.97      0.82      0.89     53366

    accuracy                           0.85     82628
   macro avg       0.75      0.89      0.79     82628
weighted avg       0.90      0.85      0.86     82628



In [ ]:

print ( 'Model accuracy %s' % accuracy_score ( y_test , y_test_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False ) )
print ( classification_report ( y_test , y_test_pred ) )

#xlabels . append ( "Gaussian" )
#ylabels . append ( accuracy_score ( y_test , y_test_pred ) ) 

Model accuracy 0.8514169876342996
accuracy normalized 105001
              precision    recall  f1-score   support

         eco       0.45      0.90      0.60      6212
         ent       0.74      0.91      0.82      9833
         int       0.60      0.87      0.71     10525
        sprt       0.99      0.91      0.95     16905
       state       0.97      0.82      0.89     79850

    accuracy                           0.85    123325
   macro avg       0.75      0.88      0.79    123325
weighted avg       0.90      0.85      0.86    123325



# **SVM classifier**

In [ ]:
import sklearn
from sklearn import svm
from sklearn . model_selection import cross_val_score

In [ ]:
smodel = svm . SVC ( C = 3, gamma = 'scale', kernel = 'rbf' )

smodel . fit ( sent_vectors , y_train )



SVC(C=3, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# *Cross validation score for svm*

In [ ]:
from sklearn import metrics
scores = cross_val_score(smodel, sent_vectors_cv, y_cv, cv=5, scoring='f1_macro')
scores

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.93 (+/- 0.01)


In [ ]:
#using k-fold

accuracy = cross_val_score(smodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10)
print(accuracy)

#get the mean of each fold 
#print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

[0.9627254  0.96623502 0.96659809 0.96405664 0.96308847 0.96284642
 0.96151519 0.96260438 0.96066328 0.96078431]


In [ ]:
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

Accuracy of Model with Cross Validation is: 96.31117202903472


The cross_validate function and multiple metric evaluation

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
#clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(smodel, sent_vectors_cv, y_cv, scoring=scoring)
sorted(scores.keys())

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

In [ ]:
 scores['test_recall_macro']

array([0.93051534, 0.93375057, 0.93037475, 0.92825006, 0.92611699])

# *accuracy score for test data*

In [ ]:
from sklearn . metrics import classification_report
y_test_pred  = smodel . predict ( sent_vectors_test )
y_cv_pred = smodel . predict(sent_vectors_cv)

In [ ]:
#print("Cross val score:",cross_val_score(Gmodel,df,data1.check,cv=5,scoring='accuracy'))
print ( 'Model accuracy %s' % accuracy_score ( y_cv , y_cv_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_cv , y_cv_pred , normalize = False ) )
print ( classification_report ( y_cv , y_cv_pred,target_names=label ) )

Model accuracy 0.9655443675267463
accuracy normalized 79781
              precision    recall  f1-score   support

         eco       0.83      0.83      0.83      4205
         ent       0.94      0.94      0.94      6778
         int       0.95      0.94      0.95      7052
        sprt       0.99      0.99      0.99     11227
       state       0.98      0.98      0.98     53366

    accuracy                           0.97     82628
   macro avg       0.94      0.94      0.94     82628
weighted avg       0.97      0.97      0.97     82628



In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_test , y_test_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False ) )
print ( classification_report ( y_test , y_test_pred ) )

Model accuracy 0.9658301236570038
accuracy normalized 119111
              precision    recall  f1-score   support

         eco       0.83      0.82      0.83      6212
         ent       0.94      0.94      0.94      9833
         int       0.95      0.94      0.95     10525
        sprt       0.99      0.99      0.99     16905
       state       0.98      0.98      0.98     79850

    accuracy                           0.97    123325
   macro avg       0.94      0.94      0.94    123325
weighted avg       0.97      0.97      0.97    123325



# **logistic regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
Lmodel = LogisticRegression ( multi_class = 'ovr' , solver = 'lbfgs' )

In [ ]:
Lmodel . fit ( sent_vectors , y_train )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## cross validation score for logistic regression 

In [ ]:
accuracy = cross_val_score(Lmodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10)
print(accuracy)

[0.95013917 0.95280165 0.95558514 0.95159143 0.95304369 0.95219654
 0.95582718 0.95376982 0.95219075 0.94916485]


In [ ]:
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

Accuracy of Model with Cross Validation is: 95.26310220937185


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
#clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(smodel, sent_vectors_cv, y_cv, scoring=scoring)
sorted(scores.keys())

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

In [ ]:
scores['test_recall_macro']

array([0.93051534, 0.93375057, 0.93037475, 0.92825006, 0.92611699])

**CLASSIFICATION** **REPORT**

In [ ]:
from sklearn . metrics import classification_report
from sklearn . model_selection import cross_val_score

y_test_pred  = Lmodel . predict ( sent_vectors_test )
y_cv_pred = Lmodel . predict(sent_vectors_cv)

In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_cv , y_cv_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_cv , y_cv_pred , normalize = False ) )
print ( classification_report ( y_cv , y_cv_pred,target_names=label ) )

Model accuracy 0.9537202885220506
accuracy normalized 78804
              precision    recall  f1-score   support

         eco       0.83      0.74      0.78      4205
         ent       0.92      0.91      0.92      6778
         int       0.92      0.92      0.92      7052
        sprt       0.99      0.98      0.98     11227
       state       0.96      0.97      0.97     53366

    accuracy                           0.95     82628
   macro avg       0.92      0.91      0.91     82628
weighted avg       0.95      0.95      0.95     82628



In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_test , y_test_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False ) )
print ( classification_report ( y_test , y_test_pred ) )

Model accuracy 0.9532860328400568
accuracy normalized 117564
              precision    recall  f1-score   support

         eco       0.82      0.74      0.78      6212
         ent       0.92      0.92      0.92      9833
         int       0.92      0.92      0.92     10525
        sprt       0.98      0.98      0.98     16905
       state       0.96      0.97      0.97     79850

    accuracy                           0.95    123325
   macro avg       0.92      0.91      0.91    123325
weighted avg       0.95      0.95      0.95    123325



# **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
Rmodel = RandomForestClassifier ( n_estimators=100 )

In [ ]:
Rmodel . fit ( sent_vectors , y_train )

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# **CROSS VALIDATION**

In [ ]:
accuracy = cross_val_score(Rmodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10)
print(accuracy)

[0.93912623 0.94215176 0.93852112 0.93852112 0.94045746 0.94057848
 0.94021542 0.94057848 0.93948197 0.93851368]


In [ ]:
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

Accuracy of Model with Cross Validation is: 93.98145709263306


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
scores = cross_validate(smodel, sent_vectors_cv, y_cv, scoring=scoring)
sorted(scores.keys())

scores['test_recall_macro']

array([0.93051534, 0.93375057, 0.93037475, 0.92825006, 0.92611699])

# **CLASSIFICATION REPORT**

In [ ]:
from sklearn . metrics import classification_report
from sklearn . model_selection import cross_val_score

y_test_pred  = Rmodel . predict ( sent_vectors_test )
y_cv_pred = Rmodel . predict(sent_vectors_cv)

In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_cv , y_cv_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_cv , y_cv_pred , normalize = False ) )
print ( classification_report ( y_cv , y_cv_pred,target_names=label ) )

Model accuracy 0.9434211163286054
accuracy normalized 77953
              precision    recall  f1-score   support

         eco       0.85      0.69      0.76      4205
         ent       0.91      0.89      0.90      6778
         int       0.91      0.85      0.88      7052
        sprt       0.99      0.97      0.98     11227
       state       0.95      0.98      0.96     53366

    accuracy                           0.94     82628
   macro avg       0.92      0.88      0.90     82628
weighted avg       0.94      0.94      0.94     82628



In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_test , y_test_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False ) )
print ( classification_report ( y_test , y_test_pred ) )

Model accuracy 0.9441475775390229
accuracy normalized 116437
              precision    recall  f1-score   support

         eco       0.84      0.69      0.76      6212
         ent       0.92      0.89      0.90      9833
         int       0.91      0.85      0.88     10525
        sprt       0.99      0.97      0.98     16905
       state       0.95      0.98      0.96     79850

    accuracy                           0.94    123325
   macro avg       0.92      0.88      0.90    123325
weighted avg       0.94      0.94      0.94    123325



# **Decission Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
Dmodel = DecisionTreeClassifier (random_state=0, max_depth=2)                

In [ ]:
Dmodel . fit ( sent_vectors , y_train )

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

# **CROSS VALIDATION**

In [ ]:
accuracy = cross_val_score(Dmodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10)
print(accuracy)

[0.80721288 0.80334019 0.80176691 0.79983057 0.80019363 0.79983057
 0.80430836 0.8039453  0.80113774 0.80440571]


In [ ]:
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

Accuracy of Model with Cross Validation is: 80.25971867778281


In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
#clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(Dmodel, sent_vectors_cv, y_cv, scoring=scoring)
sorted(scores.keys())


['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

In [ ]:
scores['test_recall_macro']

array([0.52259342, 0.51741328, 0.51368734, 0.51845153, 0.51914995])

In [ ]:
y_test_pred  = Dmodel . predict ( sent_vectors_test )
y_cv_pred = Dmodel . predict(sent_vectors_cv)

In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_cv , y_cv_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_cv , y_cv_pred , normalize = False ) )
print ( classification_report ( y_cv , y_cv_pred,target_names=label ) )

Model accuracy 0.8937043133078375
accuracy normalized 73845
              precision    recall  f1-score   support

         eco       0.62      0.61      0.61      4205
         ent       0.81      0.81      0.81      6778
         int       0.75      0.76      0.76      7052
        sprt       0.95      0.94      0.94     11227
       state       0.93      0.93      0.93     53366

    accuracy                           0.89     82628
   macro avg       0.81      0.81      0.81     82628
weighted avg       0.89      0.89      0.89     82628



In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_test , y_test_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False ) )
print ( classification_report ( y_test , y_test_pred ) )

Model accuracy 0.8052138658017434
accuracy normalized 99303
              precision    recall  f1-score   support

         eco       0.00      0.00      0.00      6212
         ent       0.65      0.79      0.71      9833
         int       0.00      0.00      0.00     10525
        sprt       0.92      0.83      0.87     16905
       state       0.81      0.97      0.88     79850

    accuracy                           0.81    123325
   macro avg       0.47      0.52      0.49    123325
weighted avg       0.70      0.81      0.75    123325



Multinomial Naivebayes(sklearn data transformation)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import Binarizer
Mmodel=make_pipeline(Binarizer(), MultinomialNB())

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(sent_vectors_train)
sent_vectors_train_transformed = scaler.transform(sent_vectors_train)

In [ ]:
Mmodel.fit(sent_vectors_train_transformed, y_train)

Pipeline(memory=None,
         steps=[('binarizer', Binarizer(copy=True, threshold=0.0)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

# CROSS VALIDATION 

In [ ]:
#using k-fold
accuracy = cross_val_score(Mmodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10)
print(accuracy)

[0.8446085  0.85320102 0.84884425 0.85053855 0.84981242 0.84884425
 0.85320102 0.84823914 0.8479787  0.85052046]


In [ ]:
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

Accuracy of Model with Cross Validation is: 84.95788272921789


In [ ]:
#using cross_validate function
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
scores = cross_validate(Mmodel, sent_vectors_cv, y_cv, scoring=scoring)
sorted(scores.keys())

scores['test_recall_macro']

array([0.87877256, 0.87991589, 0.88032083, 0.88175404, 0.8740925 ])

In [ ]:
sent_vectors_test_transformed = scaler.transform(sent_vectors_test)
y_test_pred  = Mmodel. predict ( sent_vectors_test_transformed )

In [ ]:
from sklearn import metrics
from sklearn . metrics import accuracy_score
from sklearn.metrics import classification_report


In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_test , y_test_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False ) )
print ( classification_report ( y_test , y_test_pred,target_names=label ))

Model accuracy 0.8467626190958849
accuracy normalized 104427
              precision    recall  f1-score   support

         eco       0.42      0.88      0.57      6212
         ent       0.75      0.90      0.82      9833
         int       0.62      0.84      0.71     10525
        sprt       0.98      0.93      0.95     16905
       state       0.97      0.82      0.89     79850

    accuracy                           0.85    123325
   macro avg       0.74      0.88      0.79    123325
weighted avg       0.89      0.85      0.86    123325

